In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device: ",device)
print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_capability())
print(torch.cuda.get_device_properties(device))
print(torch.backends.cudnn.is_available())
print(torch.backends.cudnn.enabled)
print(torch.backends.cudnn.deterministic)
print(torch.backends.cudnn.benchmark)
torch.cuda.init() # PyTorch'un CUDA durumunu başlatın.
torch.cuda.is_initialized() # PyTorch'un CUDA durumunun başlatılıp başlatılmadığını döndürür.

Device:  cuda
NVIDIA GeForce RTX 3050 Ti Laptop GPU
1
(8, 6)
_CudaDeviceProperties(name='NVIDIA GeForce RTX 3050 Ti Laptop GPU', major=8, minor=6, total_memory=4095MB, multi_processor_count=20)
True
True
False
False


True

 # distrübüted için bu lazım
 https://medium.com/codex/a-comprehensive-tutorial-to-pytorch-distributeddataparallel-1f4b42bb1b51

In [3]:
# Normalize training set together with augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(224, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[x / 255.0 for x in[0.507, 0.487, 0.441]],
                                     std=[x / 255.0 for x in [0.267, 0.256, 0.276]])
])

# Normalize test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[x / 255.0 for x in[0.507, 0.487, 0.441]],
                                     std=[x / 255.0 for x in [0.267, 0.256, 0.276]])
])

In [4]:
train_size = 0.8
test_size = 0.2
learning_rate=0.001
num_epochs = 30
momentum=0.9
weight_decay=1e-5
batch_size_train=64
batch_size_test=64
batch_size = 64
trainset = torchvision.datasets.ImageFolder(root = "./data",
                                         transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size_train, shuffle=True, num_workers=0)

testset = torchvision.datasets.ImageFolder(root = "./data",
                                        transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size_test, shuffle=False, num_workers=0)


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device: ",device)
print(torch.cuda.get_device_name())

Device:  cuda
NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [6]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3050 Ti Laptop GPU'

In [12]:
torch.cuda.device_count()

1

In [13]:
torch.device(device) # başka bir kontrol biçimi 

device(type='cuda')

In [14]:
torch.cuda.get_device_capability() # Sahip olduğumuz GPU'Nun CUDA yeteneğini belirten bir sayı döndürür. Örneğin benim ki 8,6

(8, 6)

In [15]:
torch.cuda.get_device_properties(device) # CUDA cihazının özelliklerini aktarır.

_CudaDeviceProperties(name='NVIDIA GeForce RTX 3050 Ti Laptop GPU', major=8, minor=6, total_memory=4095MB, multi_processor_count=20)

In [16]:
torch.cuda.init() # PyTorch'un CUDA durumunu başlatın.
torch.cuda.is_initialized() # PyTorch'un CUDA durumunun başlatılıp başlatılmadığını döndürür.

True

In [17]:

torch.cuda.synchronize() #Bir CUDA aygıtındaki tüm akışlardaki tüm çekirdeklerin tamamlanmasını bekler.

In [18]:
torch.backends.cudnn.is_available() # CUDNN'nin şu anda kullanılabilir olup olmadığını gösteren bir bool döndürür.

True

In [19]:
torch.backends.cudnn.enabled # Bir bool cuDNN etkin olup olmadığını kontrol eder.

True

In [20]:
torch.backends.cudnn.deterministic # Bir booltrue ise, sadece cuDNN neden olan, deterministic convolution algoritmaları 
#kullanılmaktadır.

False

In [21]:
torch.backends.cudnn.benchmark #Bir bool true ise, kriter convolution algoritmalarında cuDNN'i kullanır ve en hızlı olanı seçer.

False

In [17]:
#torch.cuda.set_device(1)

device0 = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Device: ",device0)
print(torch.cuda.get_device_name())



device1 = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print("Device: ",device1)
print(torch.cuda.get_device_name())

In [22]:
class BasicBlock(nn.Module):
    """Basic Block for resnet 18 and resnet 34
    """

    #BasicBlock and BottleNeck block
    #have different output size
    #we use class attribute expansion
    #to distinct
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        #residual function
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion)
        )

        #shortcut
        self.shortcut = nn.Sequential()

        #the shortcut output dimension is not the same with residual function
        #use 1*1 convolution to match the dimension
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )

    def forward(self, x):
        return nn.LeakyReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))

class BottleNeck(nn.Module):
    """Residual block for resnet over 50 layers
    """
    expansion = 2
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, stride=stride, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels * BottleNeck.expansion),
        )

        self.shortcut = nn.Sequential()

        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BottleNeck.expansion, stride=stride, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_channels * BottleNeck.expansion)
            )

    def forward(self, x):
        return nn.LeakyReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))

class ResNet(nn.Module):

    def __init__(self, block, num_block, num_classes=100):
        super().__init__()

        self.in_channels = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True))
        #we use a different inputsize than the original paper
        #so conv2_x's stride is 1
        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        """make resnet layers(by layer i didnt mean this 'layer' was the
        same as a neuron netowork layer, ex. conv layer), one layer may
        contain more than one residual block
        Args:
            block: block type, basic block or bottle neck block
            out_channels: output depth channel number of this layer
            num_blocks: how many blocks per layer
            stride: the stride of the first block of this layer
        Return:
            return a resnet layer
        """

        # we have num_block blocks per layer, the first block
        # could be 1 or 2, other blocks would always be 1
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        output = self.conv3_x(output)
        output = self.conv4_x(output)
        output = self.conv5_x(output)
        output = self.avg_pool(output)
        output = self.dropout(output)
        output = output.view(output.size(0), -1)
        output = self.fc(output)

        return output

def resnet18():
    """ return a ResNet 18 object
    """
    return ResNet(BasicBlock, [2, 2, 2, 2])

def resnet34():
    """ return a ResNet 34 object
    """
    return ResNet(BasicBlock, [3, 4, 6, 3])

def resnet50():
    """ return a ResNet 50 object
    """
    return ResNet(BottleNeck, [3, 4, 6, 3])

def resnet101():
    """ return a ResNet 101 object
    """
    return ResNet(BottleNeck, [3, 4, 23, 3])

def resnet152():
    """ return a ResNet 152 object
    """
    return ResNet(BottleNeck, [3, 8, 36, 3])

In [23]:
##from torch.nn.parallel import DistributedDataParallel as DDP ##import etme
model = resnet18() #buradan resnet modeli seçiliyor

model = model.cuda(device)# I choose ResNet50. Because of Memory :(
#model= DDP(model, device_ids=[device])

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [25]:
total = 0
print('Trainable parameters:')
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, '\t', param.numel())
        total += param.numel()
print()
print('Total', '\t', total)

Trainable parameters:
conv1.0.weight 	 1728
conv1.1.weight 	 64
conv1.1.bias 	 64
conv2_x.0.residual_function.0.weight 	 36864
conv2_x.0.residual_function.1.weight 	 64
conv2_x.0.residual_function.1.bias 	 64
conv2_x.0.residual_function.3.weight 	 36864
conv2_x.0.residual_function.4.weight 	 64
conv2_x.0.residual_function.4.bias 	 64
conv2_x.1.residual_function.0.weight 	 36864
conv2_x.1.residual_function.1.weight 	 64
conv2_x.1.residual_function.1.bias 	 64
conv2_x.1.residual_function.3.weight 	 36864
conv2_x.1.residual_function.4.weight 	 64
conv2_x.1.residual_function.4.bias 	 64
conv3_x.0.residual_function.0.weight 	 73728
conv3_x.0.residual_function.1.weight 	 128
conv3_x.0.residual_function.1.bias 	 128
conv3_x.0.residual_function.3.weight 	 147456
conv3_x.0.residual_function.4.weight 	 128
conv3_x.0.residual_function.4.bias 	 128
conv3_x.0.shortcut.0.weight 	 8192
conv3_x.0.shortcut.1.weight 	 128
conv3_x.0.shortcut.1.bias 	 128
conv3_x.1.residual_function.0.weight 	 147456
conv

In [26]:
use_gpu = True
train_acc = []
#start = time.time()
loss_list = []
running_loss = 0
total_step = len(trainloader)
for epoch in range(num_epochs):
    start = time.time()
    print("epoch basladı")
    for i, (images, labels) in enumerate(trainloader):       
        # gpu
        if use_gpu:
            if torch.cuda.is_available():
                images, labels = images.to(device), labels.to(device)
            
        outputs = model(images)
        
        loss = criterion(outputs, labels)

            
        
        # backward and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        #print("epoch bitti")
        if i % 25 == 24: # print every 500 mini batches
            print("[%d, %5d] loss: %.3f" % (epoch + 1, i+1,running_loss/25))
            running_loss = 0.0
        
        if i % 7 == 0:
            print("epoch: {} {}/{}".format(epoch,i,total_step))


    # train
    correct = 0
    total = 0
    with torch.no_grad():
        for data in trainloader:
            images, labels = data

            
            # gpu
            if use_gpu:
                if torch.cuda.is_available():
                    images, labels = images.to(device), labels.to(device)


            outputs = model(images)
            _, predicted = torch.max(outputs.data,1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print("Accuracy train %d %%"%(100*correct/total))
    train_acc.append(100*correct/total)

    # test

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data      
            # gpu
            if use_gpu:
                if torch.cuda.is_available():
                    images, labels = images.to(device), labels.to(device)                
            outputs = model(images)
            _, predicted = torch.max(outputs.data,1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print("Accuracy test %d %%"%(100*correct/total))
    train_acc.append(100*correct/total)
    loss_list.append(loss.item())
    end= time.time()
    stopWatch = end-start
    #print("epoch bitti")
    print(stopWatch)



print( "Training is done")
print('Total Training Time (second):',stopWatch)

epoch basladı
epoch: 0 0/1
Accuracy train 0 %
Accuracy test 0 %
8.552510023117065
epoch basladı
epoch: 1 0/1
Accuracy train 50 %
Accuracy test 70 %
2.1292247772216797
epoch basladı
epoch: 2 0/1
Accuracy train 70 %
Accuracy test 60 %
0.6980736255645752
epoch basladı
epoch: 3 0/1
Accuracy train 90 %
Accuracy test 90 %
0.7131409645080566
epoch basladı
epoch: 4 0/1
Accuracy train 90 %
Accuracy test 90 %
0.6984360218048096
epoch basladı
epoch: 5 0/1
Accuracy train 90 %
Accuracy test 90 %
0.7061502933502197
epoch basladı
epoch: 6 0/1
Accuracy train 100 %
Accuracy test 100 %
0.6910576820373535
epoch basladı
epoch: 7 0/1
Accuracy train 90 %
Accuracy test 100 %
0.7053089141845703
epoch basladı
epoch: 8 0/1
Accuracy train 100 %
Accuracy test 100 %
0.69138503074646
epoch basladı
epoch: 9 0/1
Accuracy train 100 %
Accuracy test 100 %
0.7085084915161133
epoch basladı
epoch: 10 0/1
Accuracy train 100 %
Accuracy test 100 %
0.6931118965148926
epoch basladı
epoch: 11 0/1
Accuracy train 100 %
Accuracy te

In [32]:
def display_learning_curves(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    ax1.plot(history.history["loss"])
    ax1.plot(history.history["val_loss"])
    ax1.legend(["train", "test"], loc="upper right")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")

    ax2.plot(history.history["acc"])
    ax2.plot(history.history["val_acc"])
    ax2.legend(["train", "test"], loc="upper right")
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Accuracy")
    plt.show()

In [33]:
display_learning_curves(history)

NameError: name 'history' is not defined